<a href="https://colab.research.google.com/github/Jatchen/Acorn_Practicum/blob/main/0208_Acorn_filterwithQuad_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
properties = pd.read_csv("/content/drive/MyDrive/properties_0207.csv")
events = pd.read_csv("/content/drive/MyDrive/events_0207.csv")

define the quad tree

In [ ]:
import math

class QuadTreeNode:
    def __init__(self, x, y, property_id):
        self.x = x
        self.y = y
        self.property_id = property_id
        self.children = [None, None, None, None]  # NE, NW, SW, SE

class QuadTree:
    def __init__(self, boundary, max_depth=10):
        self.boundary = boundary  # (min_x, min_y, max_x, max_y)
        self.root = None
        self.max_depth = max_depth

    def _insert(self, node, x, y, property_id, depth):
        if node is None:
            return QuadTreeNode(x, y, property_id)

        # Determine the quadrant for the property
        mid_x = (self.boundary[0] + self.boundary[2]) / 2
        mid_y = (self.boundary[1] + self.boundary[3]) / 2
        quadrant = (y > mid_y) << 1 | (x > mid_x)

        if depth < self.max_depth:
            node.children[quadrant] = self._insert(node.children[quadrant], x, y, property_id, depth + 1)
        return node

    def insert(self, x, y, property_id):
        self.root = self._insert(self.root, x, y, property_id, 0)

    def _query(self, node, x, y, radius, results):
        if node is None:
            return

        # Check if the current node is within the radius
        dist = math.sqrt((node.x - x)**2 + (node.y - y)**2)
        if dist <= radius:
            results.append(node.property_id)

        # Check each child node
        for child in node.children:
            if child is not None:
                self._query(child, x, y, radius, results)

    def query(self, x, y, radius):
        results = []
        self._query(self.root, x, y, radius, results)
        return results

# Convert latitude and longitude to a simple planar coordinate system
# This is a simplification and may not be accurate for large distances
def lat_lon_to_xy(lat, lon):
    # Simple conversion - not accurate for real-world distances
    x = lon * 85.0  # Roughly converts longitude to kilometers
    y = lat * 111.0  # Roughly converts latitude to kilometers
    return x, y


use

In [ ]:
events = events.head(1000) #if not do this, the dataset will be too large to run

In [ ]:
# Initialize the QuadTree with a boundary that includes all properties
boundary = (min(properties['longitude']), min(properties['latitude']),
            max(properties['longitude']), max(properties['latitude']))
qt = QuadTree(boundary)

# Insert properties into the QuadTree
for index, row in properties.iterrows():
    x, y = lat_lon_to_xy(row['latitude'], row['longitude'])
    qt.insert(x, y, row['property_id'])

# For each event, query the QuadTree for properties within 25 miles (~40.23 km)
for index, event in events.iterrows():
    # Assuming the property_id in events matches one in properties
    property_row = properties[properties['property_id'] == event['property_id']].iloc[0]
    x, y = lat_lon_to_xy(property_row['latitude'], property_row['longitude'])
    nearby_property_ids = qt.query(x, y, 40.23)  # 25 miles in kilometers
    print(f"Event {event['property_id']} nearby properties: {nearby_property_ids}")

Event 80272 nearby properties: []
Event 80024 nearby properties: []
Event 76737 nearby properties: []
Event 80113 nearby properties: []
Event 80291 nearby properties: []
Event 80538 nearby properties: []
Event 72428 nearby properties: []
Event 80317 nearby properties: []
Event 80335 nearby properties: []
Event 75800 nearby properties: []
Event 77616 nearby properties: []
Event 80132 nearby properties: []
Event 80317 nearby properties: []
Event 80425 nearby properties: []
Event 80425 nearby properties: []
Event 79530 nearby properties: []
Event 77355 nearby properties: []
Event 79579 nearby properties: []
Event 80594 nearby properties: []
Event 76690 nearby properties: []
Event 78920 nearby properties: []
Event 80436 nearby properties: []
Event 80452 nearby properties: []
Event 80078 nearby properties: []
Event 80180 nearby properties: []
Event 80261 nearby properties: []
Event 80548 nearby properties: []
Event 79595 nearby properties: []
Event 72637 nearby properties: []
Event 76735 ne